urso_dataに関する分析コード

In [1]:
cd ..

/home/is/akiyoshi-n/my-project


### 前処理

In [2]:
import re
import pandas as pd

# 正規表現パターンと対応する置換表現を辞書で定義
replacements = {
    r"@(\S*)": "[USR]", # ユーザ名を[USR]に置換
    r"http(\S*)": "[URL]", # URLを[URL]に置換
}

# 文中のマッチした箇所を対応する置換表現に置換する関数
def apply_regex(text):
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text

# テキストファイルを読み込み、1つ目と3つ目のフィールドに前処理を適用し、それらを出力する関数
def preprocess_file(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        processed_data = []
        for line in file:
            fields = line.strip().split('\t')
            # 2つ目のフィールド（前処理なし）と3つ目のフィールド（前処理あり）4つ目のフィールド（前処理なし）を取得
            first_field = fields[1] # ユーザ名
            third_field = apply_regex(fields[2]) # ツイート本文
            forth_field = fields[3] # ツイート日時
            processed_data.append([first_field, third_field, forth_field])
    return processed_data

In [3]:
from pathlib import Path
DATASET_PATH = '/home/is/akiyoshi-n/my-project/data'

In [4]:
# 関数を使用してファイルを前処理し、結果をDataFrameに格納
input_file = f'{DATASET_PATH}/urso_users.txt' # 入力ファイル名
processed_data = preprocess_file(input_file)
column_names = ['Name','text','time']
# DataFrameを作成
df = pd.DataFrame(processed_data, columns=column_names)

In [11]:
processed_data

[['468251793',
  '同意！ RT [USR] [USR] 自分は吸わないですが愛煙家の中で他人の煙は嫌いって自己中心的な方も居るそうでイラッとします。',
  '2009-11-03 00:25:04'],
 ['468251793', 'みなさんおやすみなさい～。また明日。', '2009-11-03 00:37:35'],
 ['468251793', 'おはようございます。今日は休みなので少し寝坊。', '2009-11-03 07:45:41'],
 ['468251793',
  'RT [USR] Ukraine西部Ternopil, Lviv, Ivano-Frankivsk, Chernivtsiなどで新型インフルエンザによる入院、死者急増(WHO確認)。子ども1100人以上を含む2300人超が入院、ICU利用は子ども32人を含む13',
  '2009-11-03 07:48:12'],
 ['468251793', '[USR] [USR] [USR] おはようございます。寒い！', '2009-11-03 07:50:33'],
 ['468251793',
  '伊集院光さんがtwitter始めたようです。 RT [USR] 伊集院光だ！本人？とりあえずフォロー [USR]',
  '2009-11-03 12:14:51'],
 ['468251793',
  '雑誌なんかは全部電子書籍でもいいですね。 RT [USR] 免許更新の際に自動車教習所でもらうハンドブックなど講習の際に一度しか開かないので、そういうものが電子化されるといいですねRT [USR] [USR] 例えば、日本において現在発行されている',
  '2009-11-03 12:16:20'],
 ['468251793',
  '予想通り風邪発症であります。これ以上悪化しないよう、今日は休養に専念します。',
  '2009-11-03 12:23:01'],
 ['468251793',
  'ありがとうございます。本当は悪寒、関節の違和感があったあたりで休めば良かったのですが、仕事で無理をしてしまいました。RT [USR] [USR] お大事に',
  '2009-11-03 12:28:04'],
 ['468251793',
  '上が30！よくぞ

In [11]:
import pandas as pd
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
import numpy as np

column_names = ['Num_ID','Name','text','time']
df = pd.read_csv(f"{DATASET_PATH}/urso_users.txt", sep='\t', names=column_names)

In [12]:
df

,Num_ID,Name,text,time
0,5365228407,468251793,同意！ RT @yniimi: @468251793 自分は吸わないですが愛煙家の中で他人の...,2009-11-03 00:25:04
1,5365512833,468251793,みなさんおやすみなさい～。また明日。,2009-11-03 00:37:35
2,5375233193,468251793,おはようございます。今日は休みなので少し寝坊。,2009-11-03 07:45:41
3,5375293031,468251793,"RT @TweetMedmail: Ukraine西部Ternopil, Lviv, Iva...",2009-11-03 07:48:12
4,5375348952,468251793,@Hurdy @mizu34 @mie_sama おはようございます。寒い！,2009-11-03 07:50:33
...,...,...,...,...
510264,11034722800,you1,仕事は俺と同じ夢を見てくれているんだろうか…。 RT @Consaneko: 仕事とともに泣...,2010-03-25 22:50:04
510265,11034768259,you1,@hitomi5310 いや絶対この発言するように仕向けたでしょ姐さんw,2010-03-25 22:51:05
510266,11035587060,you1,@hitomi5310 姐さんやっぱSだわ,2010-03-25 23:09:28
510267,11035699513,you1,かかか、かえるー,2010-03-25 23:12:00


In [13]:
# pickleファイルを読み込む
import pickle
with open(f"{DATASET_PATH}/urso_Dataset_type.pkl", 'rb') as f:
    df_Dataset = pickle.load(f)

In [14]:
df_Dataset['texts'][-10:]

['[USR] ちゃぼをはしごする。',
 'でもシャルケには無得点とかよくあるイメージ RT [USR] 間違いありませんｗ\u3000RT 2点取られたら4点返して終了間際に1失点みたいなイメージ＞ブレーメン (via [USR]',
 '[USR] もう じらさないーでー がまんでーきないー！',
 '熱いプレーをしてるのに、仕事が揺れてくれない(コンササポ的な意味で)',
 '[USR] どんなお店のジャンルがいいんでしょうか。和食とかフレンチとか',
 '仕事は俺と同じ夢を見てくれているんだろうか…。 RT [USR] 仕事とともに泣いて共に笑って同じ夢を見ていないんじゃないかな\u3000RT [USR] 熱いプレーをしてるのに、仕事が揺れてくれない(コンササポ的な意味で)',
 '[USR] いや絶対この発言するように仕向けたでしょ姐さんw',
 '[USR] 姐さんやっぱSだわ',
 'かかか、かえるー',
 '[USR] ぐったりおつかれ']

In [16]:
# df['text']とdf_Dataset['texts']で比較
for i in range(10):
    print(df['text'][i])
    print(df_Dataset['texts'][i])
    print()

同意！ RT @yniimi: @468251793 自分は吸わないですが愛煙家の中で他人の煙は嫌いって自己中心的な方も居るそうでイラッとします。
同意！ RT [USR] [USR] 自分は吸わないですが愛煙家の中で他人の煙は嫌いって自己中心的な方も居るそうでイラッとします。

みなさんおやすみなさい～。また明日。
みなさんおやすみなさい～。また明日。

おはようございます。今日は休みなので少し寝坊。
おはようございます。今日は休みなので少し寝坊。

RT @TweetMedmail: Ukraine西部Ternopil, Lviv, Ivano-Frankivsk, Chernivtsiなどで新型インフルエンザによる入院、死者急増(WHO確認)。子ども1100人以上を含む2300人超が入院、ICU利用は子ども32人を含む13
RT [USR] Ukraine西部Ternopil, Lviv, Ivano-Frankivsk, Chernivtsiなどで新型インフルエンザによる入院、死者急増(WHO確認)。子ども1100人以上を含む2300人超が入院、ICU利用は子ども32人を含む13

@Hurdy @mizu34 @mie_sama おはようございます。寒い！
[USR] [USR] [USR] おはようございます。寒い！

伊集院光さんがtwitter始めたようです。 RT @morishi_ss: 伊集院光だ！本人？とりあえずフォロー @HikaruIjuin
伊集院光さんがtwitter始めたようです。 RT [USR] 伊集院光だ！本人？とりあえずフォロー [USR]

雑誌なんかは全部電子書籍でもいいですね。 RT @jungalian: 免許更新の際に自動車教習所でもらうハンドブックなど講習の際に一度しか開かないので、そういうものが電子化されるといいですねRT @maruyama3: @mao3mao3 例えば、日本において現在発行されている
雑誌なんかは全部電子書籍でもいいですね。 RT [USR] 免許更新の際に自動車教習所でもらうハンドブックなど講習の際に一度しか開かないので、そういうものが電子化されるといいですねRT [USR] [USR] 例えば、日本において現在発行されている

予想通り風邪発症であります。これ以上悪化しないよう、今日は

In [17]:
df['Name'].value_counts()

Name
gadget7a        55258
c50cub96        47250
dobutas         24520
VoiceActBlog    23093
xenkichi        17713
                ...  
torichan55        370
azarashi55        353
cosume7           320
uzukko            291
dubm              269
Name: count, Length: 85, dtype: int64

In [79]:
df.loc[92380:92383][:]

,Num_ID,Name,text,time
92380,6360098140,c50cub96,1回駄目だと思ったものは駄目だよな、もう。,2009-12-05 13:14:21
92381,6360143154,c50cub96,ただし、イケメンに限…らない”女性も！ オタクを選んだ美人女性「オタクは紳士、一途で誠実、2...,2009-12-05 13:23:38
92382,6360326630,c50cub96,みんないいとこだよーｗ RT @kmz2k: 関東に出てきて長いが、日光にも鎌倉にも箱根に...,2009-12-05 13:24:07
92383,6360337424,c50cub96,仙台と札幌にも住んでみたいのう。,2009-12-05 13:24:35


In [82]:
df['text'][92381]

'ただし、イケメンに限…らない”女性も！ オタクを選んだ美人女性「オタクは紳士、一途で誠実、2次元にしか浮気をしない」    http://bit.ly/5jTwlD    ただし金持ちに限る ｗｗｗ\t2009-12-05 13:16:15\n6360170467\tc50cub96\tこっちもまだー＠墨田区 RT @wanamy: 雨なんだ？傘もって出たけど、まだ降ってない江東区。\t2009-12-05 13:17:27\n6360185626\tc50cub96\tただいま再生中： フォーチュン・クッキー - Pizzicato Five\t2009-12-05 13:18:05\n6360218942\tc50cub96\t@killingname んじゃ次は飯ですなw\t2009-12-05 13:19:31\n6360267471\tc50cub96\tｗｗｗ  ありがとうｗｗｗ  RT @tokinaka: @c50cub96 つ http://bit.ly/6ES6FE\t2009-12-05 13:21:36\n6360282769\tc50cub96\t@a2mal_ おつかれー、ゆっくり寝るんだ\t2009-12-05 13:22:16\n6360315529\tc50cub96\tあれだってメイドという偶像崇拝w RT @Ken50go: 「2次元にしか浮気をしない」って…メイドcafeは3次元だ RT @c50cub96: ただし、イケメンに限…らない”女性も！ オタクを選んだ美人女性「オタクは紳士、一途で誠実、2次元にしか浮気をしない」http://'

In [80]:
df_Dataset['texts'][92380:92384]

['1回駄目だと思ったものは駄目だよな、もう。',
 '"ただし、イケメンに限…らない”女性も！ オタクを選んだ美人女性「オタクは紳士、一途で誠実、2次元にしか浮気をしない」    [URL]    ただし金持ちに限る ｗｗｗ',
 'こっちもまだー＠墨田区 RT [USR] 雨なんだ？傘もって出たけど、まだ降ってない江東区。',
 'ただいま再生中： フォーチュン・クッキー - Pizzicato Five']

### データフレームに変換

In [6]:
import pandas as pd
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
import numpy as np
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
# tokenizerの定義
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# テキストデータを取得
texts = df['text'].values.tolist()
# テキストを含む辞書型のデータセットとして返す
test_dataset = {
    'texts': texts
}
# 辞書型をデータフレームに変換
# 最大トークン数
MAX_LEN = 128
test_dataset = preprocess_for_Trainer(test_dataset, tokenizer, max_len=MAX_LEN)

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'max_len': 128} of the transform datasets.arrow_dataset.Dataset._map_single cou

Map:   0%|          | 0/510809 [00:00<?, ? examples/s]

In [8]:
test_dataset

Dataset({
    features: ['texts', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 510809
})

In [7]:
# Datasetをpickle形式で保存
import pickle
with open(f'{DATASET_PATH}/urso_Dataset_type.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)